# Imports

In [ ]:
import pandas as pd

# Data Collection

In [ ]:
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Plastic_Classification/train_X_pc.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Plastic_Classification/train_Y_pc.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Plastic_Classification/test_X_pc.csv

--2021-09-16 16:37:21--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Plastic_Classification/train_X_pc.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.66.111
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.66.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9950 (9.7K) [text/csv]
Saving to: ‘train_X_pc.csv’

train_X_pc.csv      100%[===================>]   9.72K  --.-KB/s    in 0s      

2021-09-16 16:37:22 (126 MB/s) - ‘train_X_pc.csv’ saved [9950/9950]

--2021-09-16 16:37:22--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Plastic_Classification/train_Y_pc.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-bac

In [ ]:
train_X_df = pd.read_csv('train_X_pc.csv')
train_Y_df = pd.read_csv('train_Y_pc.csv', header=None).squeeze()
test_X_df = pd.read_csv('test_X_pc.csv')

# Understanding Data

In [ ]:
train_X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Chlorine  160 non-null    float64
 1   Hardness  160 non-null    float64
 2   Hydrogen  160 non-null    float64
 3   Sulphur   160 non-null    float64
 4   Carbon    160 non-null    float64
 5   Oxygen    160 non-null    float64
 6   Nitrogen  160 non-null    float64
dtypes: float64(7)
memory usage: 8.9 KB


In [ ]:
train_X_df.head()

,Chlorine,Hardness,Hydrogen,Sulphur,Carbon,Oxygen,Nitrogen
0,11.044,10.455700,14.047,10.165,31.585,12.700,10.405
1,10.000,10.453963,13.900,11.863,31.210,12.079,10.906
2,11.179,10.455400,14.113,10.162,31.543,12.508,10.462
3,10.534,10.454839,14.392,10.000,31.930,12.601,10.537
4,10.000,10.458217,13.306,10.000,31.924,14.488,10.225


In [ ]:
train_Y_df.head()

0    1
1    4
2    2
3    6
4    2
Name: 0, dtype: int64

# Implementation

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV

pipe = Pipeline(steps=[('scaler', RobustScaler()),
                       ('classifier', LogisticRegression(solver='sag', max_iter=2000))])

In [ ]:
param_distributions = [
              {
                'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
                'classifier': [LogisticRegression(solver='lbfgs', max_iter=15000)],
                'classifier__penalty': ['l2', 'none'],
                'classifier__multi_class': ['ovr', 'multinomial', 'auto']
              },
              {
                'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
                'classifier': [KNeighborsClassifier()],
                'classifier__n_neighbors': range(1,20),
                'classifier__p': [1, 2, 3, 4]
              }
            ]

grid_search_cv = GridSearchCV(pipe, param_grid=param_distributions, scoring='f1_weighted', refit=True, cv=5) 
grid_search_cv.fit(train_X_df, train_Y_df)
print(grid_search_cv.best_params_)

{'classifier': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=1,
                     weights='uniform'), 'classifier__n_neighbors': 1, 'classifier__p': 1, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}


In [ ]:
grid_search_cv.score(train_X_df, train_Y_df)

1.0

In [ ]:
predicted_test_Y = grid_search_cv.predict(test_X_df)
test_Y_df = pd.DataFrame(predicted_test_Y)

Writing the predicted values to CSV file

In [ ]:
pd.DataFrame(predicted_test_Y).to_csv('predicted_test_Y_pc.csv', header=None, index=False)